# Model selection using transfer learning feature extraction

This is the part 1 of three-part series:
1. `Model selection`
2. `Fine-tuning`
3. `Evaluation`

This notebook is the demonstration of choosing best model for covid-19 [dataset](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database). The original dataset has more than two classes, however this notebook is only the modified version of the original dataset and that is the binary-class classification.

All pre-trained models are imported from the [Tensorflow Hub](https://www.tensorflow.org/hub) repository, **specifically** for *Image classification* and *Image feature vector* from [tfhub.dev](https://tfhub.dev/).

The four models are:
* [Resnet50_V2](https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5)
* [InceptionV3](https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5)
* [MobilenetV3](https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5)
* [Inception_resnetV2](https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5)



## First experiment: 50% training data

This experiment is the 40-20% split from initial 80-20% data (might sound confusing, hang on for later).

The first experiment has the following data size:
* Total images - 8,285 (2,170 covid/6,115 normal)
  * Traning images - 5,522 (1,446 covid/4,076 normal)
  * Test images - 2,763 (724 covid/2,039 normal)

Before moving on we need to import dependencies.

In [1]:
import os
import random
import zipfile
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# TensorFlow
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Weights & Biases
import wandb
from wandb.keras import WandbCallback

### Loading the data

**Note:** The directory path is set according to Kaggle kernel. We have to make sure if we want to run this notebook else where then we will have to change the directory path where is it located. Another thing to remember that we have to unzip our data directory if we want to load the file other than Kaggle kernel.

The cell below is one way to demostrate how it can be unzipped if the file is running on Google Colab.

In [2]:
# # Unzip 50% data file from google drive (uncomment the cell & set the right path if using colab)
# zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/covid_ready_dataset/covid_radiography_50_percent.zip")
# zip_ref.extractall()
# zip_ref.close()

### Inspect the data

This file structure is formatted to typical format for working with images.

More specifically:
* `train` directory contains all of the images in the training dataset with subdirectories each named after a certain class (`covid` & `normal`) containing images of that class.
* `test` directory with the same structure as the `train` directory.

```
Example for file structure

covid_normal <- top level folder
└───train <- training images
│   └───covid
│   │   │   covid-1.jpg
│   │   │   covid-2.jpg
│   │   │   ...      
│   └───normal
│       │   normal-1.jpg
│       │   normal-2.jpg
│       │   ...
│   
└───test <- testing images
│   └───covid
│   │   │   covid-50.jpg
│   │   │   covid-51.jpg
│   │   │   ...      
│   └───normal
│       │   normal-50.jpg
│       │   normal-51.jpg
│       │   ...    
```

We can now inspect each of the available directories.

In [3]:
# How many images in each folder
# Walk through 50 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("../input/covid-xrays-dataset/covid_radiography_50_percent"):
  print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [4]:
# Get the class names
data_dir = pathlib.Path('../input/covid-xrays-dataset/covid_radiography_50_percent/covid_radiography_50_percent/train')
class_names = np.array([item.name for item in data_dir.glob('*')])
print(class_names)

Visualize an image for better understanding of our data.

In [5]:
# Create function to view a random image
def view_random_image(target_dir, target_class):
  # setup target directory
  target_folder = target_dir + "/" +target_class
  print(target_folder)

  # Get a random image path
  random_image = random.sample(os.listdir(target_folder), 1)

  # Read in the image and plot it using matplotlib
  img = mpimg.imread(target_folder + '/' + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis('off')
  
  print(f'Image shape: {img.shape}')  # show the shape of the image

  return img

In [6]:
# View a random image from training dataset
img = view_random_image(target_dir='../input/covid-xrays-dataset/covid_radiography_50_percent/covid_radiography_50_percent/train',
                        target_class='covid')

In [7]:
# view the img (it is a 2D array)
img

In [8]:
# View rescaled img values
img/255.

Notice the image is grayscale `(Height, Width)` but the pre-trained models are trained on rgb images `(Hight, Width, Channels)`. The models won't accept grayscale images so we have to deal with this issue. A workaround is by duplicating color channel of grayscale image 3 times when loading it to *flow_from_directory* and that is what the parameter `color_mode` is doing under the hood when we set it to `rgb` instead of `grayscale`.

Another thing to consider that none of the pre-trained models has pre-built rescaling function, so we ensure that images are rescaled.

### Creating data loaders

Now we have the data, let's use the [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) class along with the `flow_from_directory` method to load in images.

In [9]:
# Set image size
IMG_SIZE = (224, 224)

# Set up directory paths
train_dir = '../input/covid-xrays-dataset/covid_radiography_50_percent/covid_radiography_50_percent/train'
test_dir = '../input/covid-xrays-dataset/covid_radiography_50_percent/covid_radiography_50_percent/test'

# Preprocess data and rescale images
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

# Import data from directories and turn it into batches (default batch is 32)
train_data_50_percent = train_datagen.flow_from_directory(train_dir,
                                                         target_size=IMG_SIZE,  # covert images to 224 x 224
                                                         class_mode='binary',
                                                         color_mode='rgb',  # covert grayscale to rgb
                                                         seed=42)

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMG_SIZE,
                                             class_mode='binary',
                                             color_mode='rgb',  # covert grayscale to rgb
                                             seed=42)

Loading in the data we see there are 5522 images in the training dataset belonging to 2 classes (1,446 covid/4,076 normal) and 2763 images in the test set 2,763 (724 covid/2,039 normal).

### Creating models using TensorFlow Hub

TensorFlow Hub offers many state of the art models to use from the shelf, leveraging the benefits of transfer learning and after going to some research papers about model experimentation on Covid-19 images (CXR) I decide to use following models for our experiments.

One important point to notice that we choose feature vector models so we can apply feature extraction transfer learning.

Let's get the feature vector URLs of models we are going to use.


In [10]:
# MobileNetV3 feature vector
mobilenet_url = 'https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5'

# ResNet50 V2 feature vector
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'

# InceptionV3 feature vector
inception_url = 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5'

# Inception_ResnetV2
inception_resnet_url = 'https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5'

Since we're going to be comparing four models, we create a function `create_model()`. This function will take a model's TensorFlow Hub URL, instatiate a Keras Sequential model with the appropriate number of output layers and return the model.

In [11]:
# Model definition
def create_model(model_url):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.

  Returns:
    An compiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with 1 output.
  """
  # Download the pretrained model and save it as a keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,  # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=IMG_SIZE+(3,))  # since the shape now has duplicated by 3x
  
  # Create model
  model = tf.keras.Sequential([
        feature_extractor_layer,  # use the feature extraction layer as the base
        layers.Dense(1, activation='sigmoid', name='output_layer')  # create customize output layer
  ])

  return model

Now we've got a function for creating a model. It is time to compile and fit the model.

Before that we'll set our callbacks function first.

### Setting up callbacks

Before we build a model, we want to create a Keras [Callbacks](https://keras.io/api/callbacks/) to track our models building experiments.

Experiment tracking with [Weights & Biases](https://wandb.ai/site) - log the performance of multiple models and then view and compare these models in a visual way on [WandB Dashboard](https://wandb.ai/site/dashboard) (a dashboard for inspecting neural network parameters). Helpful to compare the results of different models on our data.

We first initialize our wandb run `.init`, there we need to set our hyperparameters. They're passed in as a dictionary via the `config` argument, and then become available as the `config` attribute of `wandb`.

**Note:** We will need to`pip install wandb` if it is not already installed.

In [12]:
# Initialize wandb with our project name
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={  # include hyperparameters and metadata
                     "learning_rate": 0.001,  # Adam default learning rate
                     "epochs": 5,
                     "batch_size": 32,  # default batch size
                     "loss_function": "binary_crossentropy",
                     "architecture": "MobileNetV3",
                     "dataset": "Covid-19 CXR 50 Percent Data"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize mobilenet model
tf.keras.backend.clear_session()
mobilenet_model = create_model(mobilenet_url)
mobilenet_model.summary()

# Compile the model
# Notice that we use config, so our metadata matches what gets executed
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
mobilenet_model.compile(optimizer, config.loss_function, metrics=['accuracy'])

#### MobileNetV3 (50% data)

In [13]:
# Fit the model
mobilenet_history = mobilenet_model.fit(train_data_50_percent,
                                        epochs=config.epochs,  # use config we created earlier
                                        batch_size=config.batch_size,  # same here
                                        steps_per_epoch=len(train_data_50_percent),
                                        validation_data=test_data,
                                        # go through less of the validation data so epochs are faster
                                        validation_steps=int(0.15*len(test_data)),
                                        # notice WandCallback is used as a regular callback and again we use config to set hyperparameters
                                        callbacks=[WandbCallback()])

It seems that after only 5 epochs, MobileNetV3 feature extraction model is able to achieve ~95% accuracy.

Let's check the loss curves. We create a function to plot model loss curves which will be helpful for use again for all other models as well.

In [14]:
# plot the loss curves
def plot_loss_curves(history):
  """
  Args:
    history: Takes model saved history logs
  Returns:
    Separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('epochs')
  plt.legend();

In [15]:
plot_loss_curves(mobilenet_history)

The loss is still going down and accuracy to going up, which is a good sign that the model could improve if we would have left the training for longer.

Let's evaluate the model on whole test set. We also want to indicate that our run is completed using `run.finish()`.

In [16]:
# Evaluate mobilenet on whole test set
mobilenet_loss, mobilenet_acc = mobilenet_model.evaluate(test_data)

# Experiment completed
run.finish()

#### ResNet50V2 (50% data)

We will reinitialize the callback for new experiment and recompile the model.

In [17]:
# Initialize wandb
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "binary_crossentropy",
                     "architecture": "ResNetV2",
                     "dataset": "Covid-19 CXR 50 Percent Data"
                 })

# Configure
config = wandb.config

# Initialize resnet model
tf.keras.backend.clear_session()
resnet_model = create_model(resnet_url)
resnet_model.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
resnet_model.compile(optimizer, config.loss_function, metrics=['accuracy'])

In [18]:
# Fit the model
resnet_history = resnet_model.fit(train_data_50_percent,
                                  epochs=config.epochs,
                                  batch_size=config.batch_size,
                                  steps_per_epoch=len(train_data_50_percent),
                                  validation_data=test_data,
                                  validation_steps=int(0.15*len(test_data)),
                                  callbacks=[WandbCallback()])

In [19]:
# Plot loss curves of resnet model
plot_loss_curves(resnet_history)

In [20]:
# Evaluate resnet on whole test data
resnet_loss, resnet_acc = resnet_model.evaluate(test_data)

run.finish()

#### InceptionV3 (50% data)

In [21]:
# Initialize wandb
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "binary_crossentropy",
                     "architecture": "InceptionV3",
                     "dataset": "Covid-19 CXR 50 Percent Data"
                 })

# Configure
config = wandb.config

# Initialize inception model
tf.keras.backend.clear_session()
inception_model = create_model(inception_url)
inception_model.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
inception_model.compile(optimizer, config.loss_function, metrics=['accuracy'])

In [22]:
# Fit the model
inception_history = inception_model.fit(train_data_50_percent,
                                        epochs=config.epochs,
                                        batch_size=config.batch_size,
                                        steps_per_epoch=len(train_data_50_percent),
                                        validation_data=test_data,
                                        validation_steps=int(0.15*len(test_data)),
                                        callbacks=[WandbCallback()])

In [23]:
# Plot loss curves
plot_loss_curves(inception_history)

In [24]:
# Evaluate inception on whole test data
inception_loss, inception_acc = inception_model.evaluate(test_data)

run.finish()

#### Inception_ResnetV2 (50% data)

In [25]:
# Initialize wandb with our project name
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={  # include hyperparameters and metadata
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "binary_crossentropy",
                     "architecture": "Inception_ResNetV2",
                     "dataset": "Covid-19 CXR 50 Percent Data"
                 })

# Configure
config = wandb.config

# Initialize inception_resnet model
tf.keras.backend.clear_session()
inception_resnet_model = create_model(inception_resnet_url)
inception_resnet_model.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
inception_resnet_model.compile(optimizer, config.loss_function, metrics=['accuracy'])

In [26]:
# Fit the model
inception_resnet_history = inception_resnet_model.fit(train_data_50_percent,
                                                      epochs=config.epochs,
                                                      batch_size=config.batch_size,
                                                      steps_per_epoch=len(train_data_50_percent),
                                                      validation_data=test_data,
                                                      validation_steps=int(0.15*len(test_data)),
                                                      callbacks=[WandbCallback()])

In [27]:
# Plot loss curves
plot_loss_curves(inception_resnet_model.history)

In [28]:
# Evaluate inception_resnet on whole test data
inception_resnet_loss, inception_resnet_acc = inception_resnet_model.evaluate(test_data)

run.finish()

## Second experiment: Whole training data

There is where that confusing 40-20%, 80-20% training data part come into play.

All I did in the first experiment to cut the 80% of training data to half of its length, leaving the test data unchanged because that is the whole purpose of these experiments to try different variations of training data but evaluate to the model results on same test data.

The second experiment has the following data size:
* Total images - 13,808 (3,616 covid/10,192 normal)
  * Traning images - 11,045 (2,892 covid/8,153 normal)
  * Test images - 2,763 (724 covid/2,039 normal)

That being said, we'll have to bring the data (80-20%) in and unzip it.
All the steps are same, the only differnce is this time we have larger dataset.

So, this time we'll go through all these steps quicker.

In [29]:
# # Unzip data file from google drive  (uncomment if using colab)
# zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/covid_ready_dataset/covid_radiography_all_data.zip")
# zip_ref.extractall()
# zip_ref.close()

In [30]:
# Walk through the data directory and list number of files
for dirpath, dirnames, filenames in os.walk("../input/covid-xrays-dataset/covid_radiography_all_data"):
  print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [31]:
# Set up train directory path
train_dir = '../input/covid-xrays-dataset/covid_radiography_all_data/covid_radiography_all_data/train'

# Create data generator and rescale images
train_datagen = ImageDataGenerator(rescale=1/255.)

# Load in data
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMG_SIZE,
                                               class_mode='binary',
                                               color_mode='rgb',
                                               seed=42)

Notice we don't instantiate the test data generator again, because it is the same test data that we'll have to use through out all of our experiments.

#### MobileNetV3 (All data)

In [32]:
# Initialize wandb
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "binary_crossentropy",
                     "architecture": "MobileNetV3",
                     "dataset": "Covid-19 CXR Full Data"
                 })

# Configure
config = wandb.config

# Initialize mobilenet model
tf.keras.backend.clear_session()
mobilenet_model_2 = create_model(mobilenet_url)
mobilenet_model_2.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
mobilenet_model_2.compile(optimizer, config.loss_function, metrics=['accuracy'])

In [33]:
# Fit the model
mobilenet_history_2 = mobilenet_model_2.fit(train_data,
                                          epochs=config.epochs,
                                          batch_size=config.batch_size,
                                          steps_per_epoch=len(train_data),
                                          validation_data=test_data,
                                          validation_steps=int(0.15*len(test_data)),
                                          callbacks=[WandbCallback()])

In [34]:
# Plot the loss curve
plot_loss_curves(mobilenet_history_2)

In [35]:
# Evaluate model
mobilenet_loss_2, mobilenet_acc_2 = mobilenet_model_2.evaluate(test_data)

run.finish()

#### Resnet50V2 (All data)

In [36]:
# Initialize wandb
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "binary_crossentropy",
                     "architecture": "ResNetV2",
                     "dataset": "Covid-19 CXR Full Data"
                 })

# Configure
config = wandb.config

# Initialize resnet model
tf.keras.backend.clear_session()
resnet_model_2 = create_model(resnet_url)
resnet_model_2.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
resnet_model_2.compile(optimizer, config.loss_function, metrics=['accuracy'])

In [38]:
# Fit the model
resnet_history_2 = resnet_model_2.fit(train_data,
                                    epochs=config.epochs,
                                    batch_size=config.batch_size,
                                    steps_per_epoch=len(train_data),
                                    validation_data=test_data,
                                    validation_steps=int(0.15*len(test_data)),
                                    callbacks=[WandbCallback()])

In [39]:
# Plot the loss curves
plot_loss_curves(resnet_history_2)

In [40]:
# Evaluate model
resnet_loss_2, resnet_acc_2 = resnet_model_2.evaluate(test_data)

run.finish()

#### InceptionV3 (All data)

In [41]:
# Initialize wandb
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "binary_crossentropy",
                     "architecture": "InceptionV3",
                     "dataset": "Covid-19 CXR Full Data"
                 })

# Configure
config = wandb.config

# Initialize inception model
tf.keras.backend.clear_session()
inception_model_2 = create_model(inception_url)
inception_model_2.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
inception_model_2.compile(optimizer, config.loss_function, metrics=['accuracy'])

In [42]:
# Fit the model
inception_history_2 = inception_model_2.fit(train_data,
                                            epochs=config.epochs,
                                            batch_size=config.batch_size,
                                            steps_per_epoch=len(train_data),
                                            validation_data=test_data,
                                            validation_steps=int(0.15*len(test_data)),
                                            callbacks=[WandbCallback()])

In [43]:
# Plot the loss curves
plot_loss_curves(inception_history_2)

In [44]:
# Evaluate model
inception_loss_2, inception_acc_2 = inception_model_2.evaluate(test_data)

run.finish()

#### Inception_ResNetV2 (All data)

In [45]:
# Initialize wandb
run = wandb.init(project='MobileNetV3 vs. ResNetV2 vs. InceptionV3 vs. Inception_ResNetV2',
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "binary_crossentropy",
                     "architecture": "Inception_ResNetV2",
                     "dataset": "Covid-19 CXR Full Data"
                 })

# Configure
config = wandb.config

# Initialize inception_resnet model
tf.keras.backend.clear_session()
inception_resnet_model_2 = create_model(inception_resnet_url)
inception_resnet_model_2.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
inception_resnet_model_2.compile(optimizer, config.loss_function, metrics=['accuracy'])

In [46]:
# Fit the model
inception_resnet_history_2 = inception_resnet_model_2.fit(train_data,
                                                          epochs=config.epochs,
                                                          batch_size=config.batch_size,
                                                          steps_per_epoch=len(train_data),
                                                          validation_data=test_data,
                                                          validation_steps=int(0.15*len(test_data)),
                                                          callbacks=[WandbCallback()])

In [47]:
# Plot the loss curves
plot_loss_curves(inception_resnet_history_2)

In [48]:
# Evaluate model
inception_resnet_loss_2, inception_resnet_acc_2 = inception_resnet_model_2.evaluate(test_data)

run.finish()

## Comparing results
Now we've got the results of all our models from two variation of experiments, let's compare them.

In [49]:
# Create a list of lists of model results
model_results = [['mobilenet_50_percent', mobilenet_loss, mobilenet_acc],
                 ['mobilenet_all_data', mobilenet_loss_2, mobilenet_acc_2],
                 ['resnet_50_percent', resnet_loss, resnet_acc],
                 ['resnet_all_data', resnet_loss_2, resnet_acc_2],
                 ['inception_50_percent', inception_loss, inception_acc],
                 ['inception_all_data', inception_loss_2, inception_acc_2],
                 ['inception_resent_50_percent', inception_resnet_loss, inception_resnet_acc],
                 ['inception_resnet_all_data', inception_resnet_loss_2, inception_resnet_acc_2]]

all_results = pd.DataFrame(model_results, columns=['model', 'loss', 'accuracy'])
all_results

From our experiments, it looks like `MobileNetV3` performed the best. Although `ResNet50V2` has almost identical results as `MobileNetV3` but I'm considering MobileNetV3 to go ahead for fine-tuning, because it brings efficient results with smaller architecture as compared to ResNet50V2.

Notice models overfitting was not under consideration for these experiments. The purpose of these experiments solely to pick what model is most suitable for our data.

Overfitting will be handled in the next phase which is fine-tuning with transfer learning, that will be challenging epecially when we have highly imblance data (more normal images than covid).

Let's see how it goes...